In [1]:
from moccasin import setup_notebook

setup_notebook()

In [2]:
from moccasin.config import get_active_network, Network

active_network: Network = get_active_network()
print(active_network.name)

eth-forked


In [3]:
import boa
from boa.contracts.abi.abi_contract import ABIContract
from typing import Tuple
from moccasin.config import get_active_network, Network

STARTTING_ETH_BALANCE = int(1000e18)
STARTTING_WETH_BALANCE = int(1e18)
STARTTING_USDC_BALANCE = int(100e6)  # usdc 6 decimals!


def _add_eth_balance():
    boa.env.set_balance(boa.env.eoa, STARTTING_ETH_BALANCE)


def _add_token_balance(usdc: ABIContract, weth: ABIContract):
    our_address = boa.env.eoa

    print(f"Starting balance of WETH: {weth.balanceOf(our_address)}")
    weth.deposit(value=STARTTING_WETH_BALANCE)
    print(f"Ending balance of WETH: {weth.balanceOf(our_address)}")

    print(f"Starting balance of USDC: {usdc.balanceOf(our_address)}")
    with boa.env.prank(usdc.owner()):
        usdc.updateMasterMinter(our_address)
    usdc.configureMinter(our_address, STARTTING_USDC_BALANCE)
    usdc.mint(our_address, STARTTING_USDC_BALANCE)
    print(f"Ending balance of USDC: {usdc.balanceOf(our_address)}")


def setup_script() -> Tuple[ABIContract, ABIContract, ABIContract, ABIContract]:
    print("Starting setup script...")
    active_network: Network = get_active_network()

    usdc = active_network.manifest_named("usdc")
    weth = active_network.manifest_named("weth")

    if active_network.is_local_or_forked_network():
        _add_eth_balance()
        _add_token_balance(usdc, weth)


def moccasin_main():
    setup_script()

moccasin_main()

Starting setup script...
Starting balance of WETH: 0
Ending balance of WETH: 1000000000000000000
Starting balance of USDC: 0
Ending balance of USDC: 100000000


In [4]:
usdc = active_network.manifest_named("usdc")
weth = active_network.manifest_named("weth")

In [5]:
weth.balanceOf(boa.env.eoa)
usdc.balanceOf(boa.env.eoa)

100000000

In [6]:
from moccasin.config import get_or_initialize_config, Config

config: Config = get_or_initialize_config()
config.reload()
active_network = config.get_active_network()
aavev3_pool_address_provider = active_network.manifest_named("aavev3_pool_address_provider")
pool_address = aavev3_pool_address_provider.getPool()
print(pool_address)

config.reload()
active_network = config.get_active_network()
pool_contract = active_network.manifest_named("pool", address=pool_address)

0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2


In [7]:
REFERAL_CODE = 0

def deposit(pool_contract, token, amount):
    allowed_amount = token.allowance(boa.env.eoa, pool_contract.address)
    if allowed_amount < amount:
        token.approve(pool_contract.address, amount)
    print(f"Depositing {amount} {token.name()} into Aave contract pool {pool_contract.address}")
    pool_contract.supply(token.address, amount, boa.env.eoa, REFERAL_CODE)

usdc_balance = usdc.balanceOf(boa.env.eoa)
weth_balance = weth.balanceOf(boa.env.eoa)

if usdc_balance > 0:
    deposit(pool_contract, usdc, usdc_balance)

if weth_balance > 0:
    deposit(pool_contract, weth, weth_balance)


(
    totalCollateralBase,
    totalDebtBase,
    availableBorrowsBase,
    currentLiquidationThreshold,
    ltv,
    healthFactor,
) = pool_contract.getUserAccountData(boa.env.eoa)
print(f"""User account data:
        totalCollateralBase: {totalCollateralBase}
        totalDebtBase: {totalDebtBase}
        availableBorrowsBase: {availableBorrowsBase}
        currentLiquidationThreshold: {currentLiquidationThreshold}
        ltv: {ltv}
        healthFactor: {healthFactor}
          """)

Depositing 100000000 USD Coin into Aave contract pool 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
Depositing 1000000000000000000 Wrapped Ether into Aave contract pool 0x87870Bca3F3fD6335C3F4ce8392D69350B4fA4E2
User account data:
        totalCollateralBase: 323644898399
        totalDebtBase: 0
        availableBorrowsBase: 259983946884
        currentLiquidationThreshold: 8284
        ltv: 8033
        healthFactor: 115792089237316195423570985008687907853269984665640564039457584007913129639935
          


In [8]:
# 30% USDC and 70% WETH
config.reload()
active_network = config.get_active_network()
aave_protocol_data_provider = active_network.manifest_named("aave_protocol_data_provider")
a_tokens = aave_protocol_data_provider.getAllATokens()
print(a_tokens)

[('aEthWETH', Address('0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8')), ('aEthwstETH', Address('0x0B925eD163218f6662a35e0f0371Ac234f9E9371')), ('aEthWBTC', Address('0x5Ee5bf7ae06D1Be5997A1A72006FE6C607eC6DE8')), ('aEthUSDC', Address('0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c')), ('aEthDAI', Address('0x018008bfb33d285247A21d44E50697654f754e63')), ('aEthLINK', Address('0x5E8C8A7243651DB1384C0dDfDbE39761E8e7E51a')), ('aEthAAVE', Address('0xA700b4eB416Be35b2911fd5Dee80678ff64fF6C9')), ('aEthcbETH', Address('0x977b6fc5dE62598B08C85AC8Cf2b745874E8b78c')), ('aEthUSDT', Address('0x23878914EFE38d27C4D67Ab83ed1b93A74D4086a')), ('aEthrETH', Address('0xCc9EE9483f662091a1de4795249E24aC0aC2630f')), ('aEthLUSD', Address('0x3Fe6a295459FAe07DF8A0ceCC36F37160FE86AA9')), ('aEthCRV', Address('0x7B95Ec873268a6BFC6427e7a28e396Db9D0ebc65')), ('aEthMKR', Address('0x8A458A9dc9048e005d22849F470891b840296619')), ('aEthSNX', Address('0xC7B4c17861357B8ABB91F25581E7263E08DCB59c')), ('aEthBAL', Address('0x2516E7B3F76

In [9]:
a_weth = None
a_usdc = None
for a_token in a_tokens:
    if "WETH" in a_token[0]:
        a_weth = active_network.manifest_named("weth", address=a_token[1])
    if "USDC" in a_token[0]:
        a_usdc = active_network.manifest_named("usdc", address=a_token[1])
print(a_weth)
print(a_usdc)

<weth interface at 0x4d5F47FA6A74757f35C14fD3a6Ef8E3C9BC514E8>
<usdc interface at 0x98C23E9d8f34FEFb1B7BD6a91B7FF122F4e16F5c>


In [10]:
# Get how much they are worth to figure out if we need rebalance 30% USDC and 70% WETH
a_usdc_balance = a_usdc.balanceOf(boa.env.eoa) # 6 decimals
a_weth_balance = a_weth.balanceOf(boa.env.eoa) # 18 decimals

a_usdc_balance_normalized = a_usdc_balance / 1e6
a_weth_balance_normalized = a_weth_balance / 1e18
print(a_usdc_balance_normalized)
print(a_weth_balance_normalized)

100.000001
1.0


In [11]:
# Get price feed on chain with chainlink
config.reload()

def get_price(feed_name: str) -> float:
    active_network = get_active_network()
    price_feed = active_network.manifest_named(feed_name)
    price = price_feed.latestAnswer()
    decimals = price_feed.decimals()
    decimals_normalized = 10 ** decimals
    return price / decimals_normalized

usdc_price = get_price("usdc_usd")
weth_price = get_price("eth_usd")
print(usdc_price)
print(weth_price)

0.99998983
3136.45


In [12]:
usdc_value = a_usdc_balance_normalized * usdc_price
weth_value = a_weth_balance_normalized * weth_price
total_value = usdc_value + weth_value

target_usd_value = 0.3
target_weth_value = 0.7

usdc_percent_allocation = usdc_value / total_value
weth_percent_allocation = weth_value / total_value

BUFFER = 0.1

needs_rebalancing = (
    abs(usdc_percent_allocation - target_usd_value) > BUFFER 
    or abs(weth_percent_allocation - target_weth_value) > BUFFER
)

print(needs_rebalancing)
print(usdc_percent_allocation)
print(weth_percent_allocation)

True
0.03089774765316991
0.9691022523468301


In [13]:
a_weth.approve(pool_contract.address, a_weth.balanceOf(boa.env.eoa))
pool_contract.withdraw(weth.address, a_weth.balanceOf(boa.env.eoa), boa.env.eoa)

def print_token_balances():
  print(f"USDC balance: {usdc.balanceOf(boa.env.eoa)}")
  print(f"WETH balance: {weth.balanceOf(boa.env.eoa)}")
  print(f"aUSDC balance: {a_usdc.balanceOf(boa.env.eoa)}")
  print(f"aWETH balance: {a_weth.balanceOf(boa.env.eoa)}")
  

print_token_balances()

USDC balance: 0
WETH balance: 1000000000000000000
aUSDC balance: 100000001
aWETH balance: 0
